## Get info of each game
Here are some examples of responded API:
- https://api.rawg.io/api/games/rimworld
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

In [2]:
import csv
import requests
import json
from pprint import pprint
from time import time
import concurrent.futures
import functools
import os

### Load CSV file which has game's id and its name

In [2]:
csv_data = []
with open("data/game_id.csv", "r") as f:
    csv_data = list(csv.reader(f))
for i, val in enumerate(csv_data):
    print(val)
    if i==10: break

['3498', 'grand-theft-auto-v']
['4200', 'portal-2']
['3328', 'the-witcher-3-wild-hunt']
['5286', 'tomb-raider']
['5679', 'the-elder-scrolls-v-skyrim']
['12020', 'left-4-dead-2']
['802', 'borderlands-2']
['4062', 'bioshock-infinite']
['13536', 'portal']
['3439', 'life-is-strange-episode-1-2']
['4291', 'counter-strike-global-offensive']


## Multithreading

### This function is responsible for requesting each game and save as a JSON file in `/data/game_info/`

In [3]:
def threading(start_index, games_per_worker, urls, downloaded_files, headers):
    for url in urls[start_index : start_index + games_per_worker]:
        if url.rsplit("/")[-1] in downloaded_files: continue
        try:
            # Request API
            json_data = json.loads(requests.get(url, headers=headers).text)

            # Only include wanted keys
            D = {k:v for k,v in json_data.items() if k in include}

            # Clean up dictionary
            D["platforms"] = []
            for platform in json_data["platforms"]:
                D["platforms"].append(platform["platform"]["name"])

            for key in ("developers", "genres", "publishers", "tags"):
                D[key] = []
                for data in json_data[key]:
                    D[key].append(data["name"])

            if json_data["esrb_rating"]:
                D["esrb_rating"] = json_data["esrb_rating"]["name"]
            
            # Save as JSON file
            name = D["id"]
            with open(f"data/game_info/{name}.json","w", encoding="utf-8") as f:
                json.dump(D, f)
        except:
            print(f"Error with {url}")

#### Create folder if not existed

In [4]:
import os
if not os.path.exists('data/game_info/'):
    os.makedirs('data/game_info/')

#### Threading Preparation

In [5]:
headers = { 'User-Agent': 'App Name: Education purpose',}
include = {      
           "id",
           "slug",
           "name",
           "metacritic",
           "metacritic_platforms",
           "released",
           "tba",
           "updated",
           "website",
           "rating",
           "rating_top",
           "ratings",
           "reactions",
           "added",
           "added_by_status",
           "playtime",
           "screenshots_count",
           "movies_count",
           "creators_count",
           "achievements_count",
           "parent_achievements_count",
           "reddit_count",
           "twitch_count",
           "youtube_count",
           "reviews_count",
           "reviews_text_count",
           "ratings_count",
           "suggestions_count",
           "parents_count",
           "additions_count",
           "game_series_count",
           "esrb_rating",
           "platforms",
            "developers",
            "genres",
            "publishers",
            "esrb_rating",
            "tags"   
          }

# Set up number of workers
max_workers = 100
start_game_index = 0 # min: 0
end_game_index = len(csv_data) # max: 345707
number_of_games = end_game_index - start_game_index
games_per_worker = int(number_of_games/max_workers) 
start_index = range(start_game_index, end_game_index, games_per_worker)

# Make urls
base_url = "https://api.rawg.io/api/games/"
key = ['?key=13961a85724f47818f318bebae6b82e1' for i in range(len(csv_data)) ]
urls = [base_url + csv_data[i][0] +key[i] for i in range(len(csv_data))]

In [8]:
# Skipped downloaded files
downloaded_files = {file.split(".",1)[0] for file in os.listdir("data/game_info/")}

# Time
t0 = time()
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(threading,
                             games_per_worker=games_per_worker,
                             urls=urls,
                             downloaded_files=downloaded_files,
                             headers=headers,
                            )
    executor.map(temp, start_index)
    
# Time diff
print(f"Time taken: {time()-t0}")

Error with https://api.rawg.io/api/games/440682?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/444899?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/34532?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/267083?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/28042?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/492681?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/231972?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/411996?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/3533?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/517088?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/490845?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/490844?key=13961a85724f47818

Error with https://api.rawg.io/api/games/264548?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/375500?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/27437?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/360562?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/181388?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/385022?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/149304?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/136191?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/264547?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/25762?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/375499?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/360561?key=13961a85724f4781

Error with https://api.rawg.io/api/games/389599?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/380238?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/385014?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/360554?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/394033?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/512557?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/136184?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/503990?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/264538?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/181378?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/482235?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/117786?key=13961a85724f4

Error with https://api.rawg.io/api/games/503985?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/375447?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/490814?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/385011?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/269992?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/140656?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/300713?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/380233?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/264531?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/512553?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/394029?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/117781?key=13961a85724f4

Error with https://api.rawg.io/api/games/375427?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/44212?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/482226?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/469102?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/122293?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/394025?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/360546?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/280262?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/269986?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/149288?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/264527?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/512550?key=13961a85724f4781

...

Error with https://api.rawg.io/api/games/360498?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/88686?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/94793?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/190221?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/90109?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/429466?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/242775?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/508103?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/316897?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/217368?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/149226?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/109128?key=13961a85724f4781

Error with https://api.rawg.io/api/games/242773?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/473326?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/158372?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/89849?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/429464?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/300646?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/109126?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/181331?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/149224?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/122244?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/217366?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/162819?key=13961a85724f47

Error with https://api.rawg.io/api/games/482173?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/207846?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/54224?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/57658?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/490766?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/280204?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/7013?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/88676?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/230700?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/285332?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/140553?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/35922?key=13961a85724f47818f3

Error with https://api.rawg.io/api/games/389509?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/424773?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/259352?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/212273?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/94789?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/360494?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/51737?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/40714?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/310417?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/167475?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/308417?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/190223?key=13961a85724f4781

Error with https://api.rawg.io/api/games/508099?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/455877?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/230695?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/89222?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/473321?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/54177?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/57641?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/181326?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/88670?key=13961a85724f47818f318bebae6b82e1

Error with https://api.rawg.io/api/games/6947?key=13961a85724f47818f318bebae6b82e1
Error with https://api.rawg.io/api/games/109120?key=13961a85724f47818f318bebae6b82e1Error with https://api.rawg.io/api/games/253816?key=13961a85724f47818f31